In [3]:
# https://openalex.org/

from pyalex import Works, Authors, Sources, Institutions, Concepts, Publishers, Funders
import pyalex
pyalex.config.email = "maximilian.noichl@uni-bamberg.de"

from itertools import chain
import tqdm.auto as tqdm

from compress_pickle import dump, load



In [ ]:
from pyalex import Works
import pandas as pd



# Create a query with the specified search terms
query0 = Works().search(['"Personality" AND "Life outcomes"']
)

query = Works().search(['Perceptron']).filter(publication_year="<1979")
records= []
for record in chain(*query.paginate(per_page=200,n_max=None)):
    records.append(record)
    #print(len(records))



In [ ]:
df = pd.DataFrame(records)
print(len(df))
df.head()

727


,id,doi,title,display_name,relevance_score,publication_year,publication_date,ids,language,primary_location,...,versions,referenced_works_count,referenced_works,related_works,ngrams_url,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date
0,https://openalex.org/W2040870580,https://doi.org/10.1037/h0042519,The perceptron: A probabilistic model for info...,The perceptron: A probabilistic model for info...,6302.6006,1958,1958-01-01,{'openalex': 'https://openalex.org/W2040870580...,en,"{'is_oa': False, 'landing_page_url': 'https://...",...,[],15,"[https://openalex.org/W1570445666, https://ope...","[https://openalex.org/W4387297750, https://ope...",https://api.openalex.org/works/W2040870580/ngrams,None,https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 218}, {'year...",2024-05-24T22:42:23.907682,2016-06-24
1,https://openalex.org/W2086789740,None,Perceptrons: An Introduction to Computational ...,Perceptrons: An Introduction to Computational ...,3869.5200,1969,1969-01-01,{'openalex': 'https://openalex.org/W2086789740...,en,"{'is_oa': False, 'landing_page_url': 'http://c...",...,[],0,[],"[https://openalex.org/W3207342693, https://ope...",https://api.openalex.org/works/W2086789740/ngrams,"{'Cambridge,': [0], 'Mass.:': [1], 'MIT': [2],...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 13}, {'year'...",2024-06-01T23:12:45.442795,2016-06-24
2,https://openalex.org/W2008353316,https://doi.org/10.2307/1419730,Principles of Neurodynamics: Perceptrons and t...,Principles of Neurodynamics: Perceptrons and t...,2525.0693,1963,1963-12-01,{'openalex': 'https://openalex.org/W2008353316...,en,"{'is_oa': False, 'landing_page_url': 'https://...",...,[],0,[],"[https://openalex.org/W2931662336, https://ope...",https://api.openalex.org/works/W2008353316/ngrams,"{'Abstract': [0], ':': [1], 'Part': [2, 104, 1...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 5}, {'year':...",2024-05-25T00:29:51.038021,2016-06-24
3,https://openalex.org/W4233045210,https://doi.org/10.21236/ad0256582,PRINCIPLES OF NEURODYNAMICS. PERCEPTRONS AND T...,PRINCIPLES OF NEURODYNAMICS. PERCEPTRONS AND T...,2207.9346,1961,1961-03-15,{'openalex': 'https://openalex.org/W4233045210...,en,"{'is_oa': False, 'landing_page_url': 'https://...",...,[],0,[],"[https://openalex.org/W3194278305, https://ope...",https://api.openalex.org/works/W4233045210/ngrams,None,https://api.openalex.org/works?filter=cites:W4...,"[{'year': 2024, 'cited_by_count': 25}, {'year'...",2024-05-24T23:47:28.756377,2022-05-12
4,https://openalex.org/W2056003475,https://doi.org/10.1103/revmodphys.34.123,The Perceptron: A Model for Brain Functioning. I,The Perceptron: A Model for Brain Functioning. I,1314.0599,1962,1962-01-01,{'openalex': 'https://openalex.org/W2056003475...,en,"{'is_oa': False, 'landing_page_url': 'https://...",...,[],29,"[https://openalex.org/W118296737, https://open...","[https://openalex.org/W54078636, https://opena...",https://api.openalex.org/works/W2056003475/ngrams,None,https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 1}, {'year':...",2024-05-22T06:54:39.787596,2016-06-24


In [7]:
import networkx as nx
import json


# Create a directed graph
G = nx.DiGraph()

# Add nodes and edges
for ix,item in df.iterrows():
    G.add_node(item["id"])  # Add each work as a node
    for related in item["related_works"]:
        G.add_edge(item["id"], related)  # Create an edge from this work to each related work

# Optionally, you can print the nodes and edges to verify
print("Nodes in the network:")
print(len(G.nodes()))
print("Edges in the network:")
print(len(G.edges()))

# If you need to visualize the network
import matplotlib.pyplot as plt

nx.draw(G, with_labels=True)
plt.show()

Nodes in the network:
5174
Edges in the network:
7471
